<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>



# Demo 9.3: Web Scraping

INSTRUCTIONS:

- Run the cells
- Observe and understand the results
- Answer the questions

# Web Scraping in Python (using BeautifulSoup)

# Basics HTML
Before starting with the code, let’s understand the basics of HTML and some rules of scraping.

## HTML tags
Below is the source code for a simple HTML webpage.

    <!DOCTYPE html>  
    <html>  
        <head>
        </head>
        <body>
            <h1> First Scraping </h1>
            <p> Hello World </p>
        <body>
    </html>
    
This is the basic syntax of an HTML webpage. Every `<tag>` serves a block inside the webpage:
1. `<!DOCTYPE html>` HTML documents must start with a type declaration.
2. The HTML document is contained between `<html>` and `</html>`.
3. The meta and script declaration of the HTML document is between `<head>` and `</head>`.
4. The visible part of the HTML document is between `<body>` and `</body>` tags.
5. Title headings are defined with the `<h1>` through `<h6>` tags.
6. Paragraphs are defined with the `<p>` tag.

Other useful tags include `<a>` for hyperlinks, `<table>` for tables, `<tr>` for table rows, and `<td>` for table columns.

Also, HTML tags sometimes come with `id` or `class` attributes. The `id` attribute specifies a unique id for an HTML tag and the value must be unique within the HTML document. The `class` attribute is used to define equal styles for HTML tags with the same class. We can make use of these ids and classes to help us locate the data we want.

## Scraping Rules
1. **Always** check a website’s **Terms and Conditions** before you scrape it. Be careful to read the statements about legal use of data. Usually, the retrieved data should not be used for commercial purposes.
2. **Do not** request data from the website too aggressively with a program (also known as spamming), as this may break the website. Make sure the program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.
3. The layout of a website may change from time to time, so make sure to revisit the site and rewrite the code as needed

## Inspecting the Page
Let’s take one page from the **Memory Alpha** website as an example.

To investigate some relationships let's get the links from this page.

Open the web page on [Prinadora](http://memory-alpha.wikia.com/wiki/Prinadora) with the browser and inspect it.

Hover the cursor on the text and follow the shaded box surrounding the main text.

From the result, check the main text inside a few levels of HTML tags, which is `<section id="WikiaPage" ...>` → `<div class="WikiaPageContentWrapper">` → `<article id="WikiaMainContent" class="WikiaMainContent">`.

The unique location of the data is now known.

In [17]:
## Import Libraries
import regex as re

from urllib.parse import unquote
import urllib3
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### Define the content to retrieve (webpage's URL)

In [18]:
# specify the url
quote_page = 'https://edition.cnn.com/search?size=50&q=*&sort=newest&from=0&page=1'

### Retrieve the page
- Require Internet connection

In [19]:
# query the website and return the html to the variable ‘page’
http = urllib3.PoolManager()
r = http.request('GET', quote_page)
if r.status == 200:
    page = r.data
    print('Type of the variable \'page\':', page.__class__.__name__)
    print('Page Retrieved. Request Status: %d, Page Size: %d' % (r.status, len(page)))
else:
    print('Some problem occurred. Request Status: %s' % r.status)

Type of the variable 'page': bytes
Page Retrieved. Request Status: 200, Page Size: 1115757


### Convert the stream of bytes into a BeautifulSoup representation

In [4]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, 'html.parser')
print('Type of the variable \'soup\':', soup.__class__.__name__)

Type of the variable 'soup': BeautifulSoup


### Check the content
- The HTML source
- Includes all tags and scripts
- Can be long!

In [5]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Prinadora | Memory Alpha | Fandom
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Prinadora","wgTitle":"Prinadora","wgCurRevisionId":2694380,"wgRevisionId":2694380,"wgArticleId":8581,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Ferengi"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November",

### Check the HTML's Title

In [6]:
print('Title tag :%s:' % soup.title)
print('Title text:%s:' % soup.title.string)

Title tag :<title>Prinadora | Memory Alpha | Fandom</title>:
Title text:Prinadora | Memory Alpha | Fandom:


### Wikimedia's article
- Wiki pages use the tag `article` for the actual content of the page

        <article class="WikiaMainContent" id="WikiaMainContent">
            <div class="WikiaMainContentContainer" id="WikiaMainContentContainer">
                <div class="WikiaArticle" id="WikiaArticle">

In [7]:
tag = 'article'
article = soup.find_all(tag)[0]
print('Type of the variable \'article\':', article.__class__.__name__)

Type of the variable 'article': Tag


### Get some of the text
- Plain text without HTML tags

In [8]:
# show the first 500 characters after removing redundant newlines
print(re.sub(r'\n\n+', '\n', article.text)[:500])


Species:
Ferengi
Status:
Active (2373)
Marital Status:
Divorced and in another relationship
Spouse(s):
Rom (ex)
Children:
One son, Nog
Prinadora was a Ferengi female, the first wife of Rom and mother of Nog. 
Rom's first marriage began when he signed a standard five-year marriage contract with Prinadora's father to produce a child. He then made the mistake of falling in love with his wife. As a result, he signed an extension to the contract without reading the fine print, allowing Prinadora's f


### Find the links in the text

In [9]:
# identify the type of tag to retrieve
tag = 'a'
# create a list with the links from the `<a>` tag
tag_list = [t.get('href') for t in article.find_all(tag)]
tag_list

['/wiki/Ferengi',
 '/wiki/2373',
 '/wiki/Divorce',
 '/wiki/Prinadora%27s_richer_man_001',
 '/wiki/Rom',
 '/wiki/Son',
 '/wiki/Nog',
 '/wiki/Ferengi',
 '/wiki/Female',
 '/wiki/Wife',
 '/wiki/Rom',
 '/wiki/Mother',
 '/wiki/Nog',
 '/wiki/Marriage',
 None,
 '/wiki/Prinadora%27s_father_001',
 '/wiki/Child',
 '/wiki/Love',
 '/wiki/Prinadora%27s_richer_man_001',
 '/wiki/Star_Trek:_Deep_Space_Nine',
 '/wiki/Doctor_Bashir,_I_Presume_(episode)',
 '/wiki/2373',
 '/wiki/Leeta',
 '/wiki/Star_Trek:_Deep_Space_Nine',
 '/wiki/Ferengi_Love_Songs_(episode)',
 '#Appendices',
 '#Background_information',
 '#Apocrypha',
 '#External_link',
 '/wiki/DS9_Season_6',
 '/wiki/Change_of_Heart_(episode)',
 '/wiki/Star_Trek:_Deep_Space_Nine_Companion',
 '/wiki/Lolita_Fatjo',
 '/wiki/Max_Grodenchik',
 '/wiki/Aron_Eisenberg',
 '/wiki/Chase_Masterson',
 '/wiki/Change_of_Heart_(episode)',
 '/wiki/Deep_Space_9',
 'http://www.treknation.com/articles/a_weekend_with_the_stars_part_one.shtml',
 '/wiki/Worlds_of_Star_Trek:_Dee

In [10]:
# keep only the links to the wiki itself
tag_list = [t[6:] for t in tag_list if (t) and (t.startswith('/wiki/'))]
tag_list

['Ferengi',
 '2373',
 'Divorce',
 'Prinadora%27s_richer_man_001',
 'Rom',
 'Son',
 'Nog',
 'Ferengi',
 'Female',
 'Wife',
 'Rom',
 'Mother',
 'Nog',
 'Marriage',
 'Prinadora%27s_father_001',
 'Child',
 'Love',
 'Prinadora%27s_richer_man_001',
 'Star_Trek:_Deep_Space_Nine',
 'Doctor_Bashir,_I_Presume_(episode)',
 '2373',
 'Leeta',
 'Star_Trek:_Deep_Space_Nine',
 'Ferengi_Love_Songs_(episode)',
 'DS9_Season_6',
 'Change_of_Heart_(episode)',
 'Star_Trek:_Deep_Space_Nine_Companion',
 'Lolita_Fatjo',
 'Max_Grodenchik',
 'Aron_Eisenberg',
 'Chase_Masterson',
 'Change_of_Heart_(episode)',
 'Deep_Space_9',
 'Worlds_of_Star_Trek:_Deep_Space_Nine,_Volume_Three',
 'Worlds_of_Star_Trek:_Deep_Space_Nine',
 'Grand_Nagus',
 'Ro_Laren',
 'Bajoran',
 'Special:Categories',
 'Category:Ferengi']

In [11]:
# create a filter for undesired links
filter  = '(%s)' % '|'.join([
    'Category:',
    'File:',
    'Help:',
    'Memory_Alpha:',
    'Portal:',
    'action=',
    'Special:',
    'Star_Trek:',
    'Star_Trek_',
    'Talk:'
])
# remove the links that are found in the filter
tag_list = [t for t in tag_list if not re.search(filter, t)]
tag_list

['Ferengi',
 '2373',
 'Divorce',
 'Prinadora%27s_richer_man_001',
 'Rom',
 'Son',
 'Nog',
 'Ferengi',
 'Female',
 'Wife',
 'Rom',
 'Mother',
 'Nog',
 'Marriage',
 'Prinadora%27s_father_001',
 'Child',
 'Love',
 'Prinadora%27s_richer_man_001',
 'Doctor_Bashir,_I_Presume_(episode)',
 '2373',
 'Leeta',
 'Ferengi_Love_Songs_(episode)',
 'DS9_Season_6',
 'Change_of_Heart_(episode)',
 'Lolita_Fatjo',
 'Max_Grodenchik',
 'Aron_Eisenberg',
 'Chase_Masterson',
 'Change_of_Heart_(episode)',
 'Deep_Space_9',
 'Grand_Nagus',
 'Ro_Laren',
 'Bajoran']

In [12]:
# remove duplicates
tag_list = list(set(tag_list))
tag_list

['Aron_Eisenberg',
 'Chase_Masterson',
 'Prinadora%27s_father_001',
 'Doctor_Bashir,_I_Presume_(episode)',
 'Deep_Space_9',
 'Grand_Nagus',
 '2373',
 'Female',
 'Ferengi',
 'Mother',
 'Lolita_Fatjo',
 'Change_of_Heart_(episode)',
 'Love',
 'Ferengi_Love_Songs_(episode)',
 'Divorce',
 'DS9_Season_6',
 'Leeta',
 'Marriage',
 'Max_Grodenchik',
 'Ro_Laren',
 'Child',
 'Bajoran',
 'Prinadora%27s_richer_man_001',
 'Wife',
 'Son',
 'Nog',
 'Rom']

In [13]:
# convert escaped sequences
tag_list = [unquote(t) for t in tag_list]
tag_list

['Aron_Eisenberg',
 'Chase_Masterson',
 "Prinadora's_father_001",
 'Doctor_Bashir,_I_Presume_(episode)',
 'Deep_Space_9',
 'Grand_Nagus',
 '2373',
 'Female',
 'Ferengi',
 'Mother',
 'Lolita_Fatjo',
 'Change_of_Heart_(episode)',
 'Love',
 'Ferengi_Love_Songs_(episode)',
 'Divorce',
 'DS9_Season_6',
 'Leeta',
 'Marriage',
 'Max_Grodenchik',
 'Ro_Laren',
 'Child',
 'Bajoran',
 "Prinadora's_richer_man_001",
 'Wife',
 'Son',
 'Nog',
 'Rom']

In [14]:
# convert underscore to space
tag_list = [re.sub('_', ' ', t) for t in tag_list]
tag_list

['Aron Eisenberg',
 'Chase Masterson',
 "Prinadora's father 001",
 'Doctor Bashir, I Presume (episode)',
 'Deep Space 9',
 'Grand Nagus',
 '2373',
 'Female',
 'Ferengi',
 'Mother',
 'Lolita Fatjo',
 'Change of Heart (episode)',
 'Love',
 'Ferengi Love Songs (episode)',
 'Divorce',
 'DS9 Season 6',
 'Leeta',
 'Marriage',
 'Max Grodenchik',
 'Ro Laren',
 'Child',
 'Bajoran',
 "Prinadora's richer man 001",
 'Wife',
 'Son',
 'Nog',
 'Rom']

In [15]:
# order the list
tag_list.sort()
tag_list

['2373',
 'Aron Eisenberg',
 'Bajoran',
 'Change of Heart (episode)',
 'Chase Masterson',
 'Child',
 'DS9 Season 6',
 'Deep Space 9',
 'Divorce',
 'Doctor Bashir, I Presume (episode)',
 'Female',
 'Ferengi',
 'Ferengi Love Songs (episode)',
 'Grand Nagus',
 'Leeta',
 'Lolita Fatjo',
 'Love',
 'Marriage',
 'Max Grodenchik',
 'Mother',
 'Nog',
 "Prinadora's father 001",
 "Prinadora's richer man 001",
 'Ro Laren',
 'Rom',
 'Son',
 'Wife']

### Create a filter for unwanted types of articles

In [16]:
filter  = '(%s)' % '|'.join([
    'episode',
    'lternate_reality', # both Alternate_reality and alternate_reality
    'mirror',
    'rank',
    'production',
    'Season'
])
# remove the links that are found in the filter
tag_list = [t for t in tag_list if not re.search(filter, t)]
tag_list

['2373',
 'Aron Eisenberg',
 'Bajoran',
 'Chase Masterson',
 'Child',
 'Deep Space 9',
 'Divorce',
 'Female',
 'Ferengi',
 'Grand Nagus',
 'Leeta',
 'Lolita Fatjo',
 'Love',
 'Marriage',
 'Max Grodenchik',
 'Mother',
 'Nog',
 "Prinadora's father 001",
 "Prinadora's richer man 001",
 'Ro Laren',
 'Rom',
 'Son',
 'Wife']



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



